# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time to get down to business.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between a challenging and approachable project. Let's begin.

# Load and Examine Data

In [2443]:
# These imports are a mess. Should just import sklearn.linear_model instead
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

In [2444]:
cars = pd.read_csv("data/used_cars.csv")
print(len(cars))

4009


Let's just get a quick overview of some basic precleaning stuff.

In [2445]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [2446]:
print(f"Missing Values:\n{cars.isna().sum()}\n\nTotal Duplicates:\n{cars.duplicated().sum()}")

Missing Values:
brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

Total Duplicates:
0


Now that that's out of the way, it's time to actually take a peak at the data.

In [2447]:
cars.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


# Clean the Data

Now that the preliminary stuff is out of the way, let's start cleaning the data up.

It'd be best to start simple here and worry about the basic numeric conversion first. We'll start by removing unnecessary string values from columns that will be designated as strictly numeric. I also think it'd be interesting to see if there are any differences in the model outputs when using `float32` instead of `float64`. We'll get to that much later though.

For now, let's start with simple data cleaning. Then we can chip away at the more tricky regex stuff and increase the complexity as we get a feel for the data.

**Note: We can definitely work on making this SIGNIFICANTLY cleaner by aggregating a lot of this into less lines of code. We want BEST PRACTICES AND THIS IS NOT THAT.**

In [2448]:
# Bugatti cars serve as a major outlier. Don't include
# Also get rid of smart
cars = cars[~cars["brand"].isin(["Bugatti", "smart"])]

# Remove non integer string values
cars[["milage", "price"]] = cars[["milage", "price"]].replace(r'[^\d]', '', regex=True)    

# `engine` column
cars["horsepower"] = cars["engine"].str.extract(r'([\d.]+)HP', expand=False)
cars["liters"] = cars["engine"].str.extract(r'([\d.]+)L', expand=False)

# `fuel_type` column
cars["gas_diesel_other"] = (
    cars["fuel_type"].str.contains("gasoline", case=False, na=False) |
    cars["fuel_type"].str.contains("diesel", case=False, na=False) |
    cars["fuel_type"].str.contains("fuel", case=False, na=False)
)
cars["hybrid"] = cars["fuel_type"].str.contains("hybrid", case=False, na=False)
cars["electric"] = cars["fuel_type"].isna()
print(f"Conflicting fuel type values? {(cars[['gas_diesel_other', 'hybrid', 'electric']].sum(axis=1) > 1).any()}")

# `transmission` column
cars["automatic_transmission"] = (
    cars["transmission"].str.contains("A/T", case=False) |
    cars["transmission"].str.contains("AT", case=False) |
    cars["transmission"].str.contains("auto", case=False) |
    cars["transmission"].str.contains("automatic", case=False) |
    cars["transmission"].str.contains("cvt", case=False)
)
cars['dual_shift_transmission'] = cars['transmission'].str.contains("dual", case=False)
cars["manual_transmission"] = (
    cars['transmission'].str.contains("manual", case=False) |
    cars['transmission'].str.contains("m/t", case=False) |
    cars['transmission'].str.contains("mt", case=False)
)
cars.loc[cars['dual_shift_transmission'], ['automatic_transmission', 'manual_transmission']] = False
cars.loc[cars['manual_transmission'], ['automatic_transmission', 'dual_shift_transmission']] = False
mask = ~(cars[['automatic_transmission', 'dual_shift_transmission', 'manual_transmission']] == False).all(axis=1) # Remove rows with `False` in all three newly created transmission columns
cars = cars[mask]
print(f"Conflicting transmission values? {(cars[['automatic_transmission', 'manual_transmission', "dual_shift_transmission"]].sum(axis=1) > 1).any()}")

# `accident` and `clean_title` columns
cars["accident_bool"] = cars["accident"] != "None reported"
cars.dropna(subset=['accident'], inplace=True)

# Deletions
cars.drop(["fuel_type", "engine", "transmission", "accident", "clean_title", "int_col", "ext_col"], axis=1, inplace=True)
cars.head(10)

Conflicting fuel type values? False
Conflicting transmission values? False


,brand,model,model_year,milage,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,manual_transmission,accident_bool
0,Ford,Utility Police Interceptor Base,2013,51000,10300,300.0,3.7,True,False,False,True,False,False,True
1,Hyundai,Palisade SEL,2021,34742,38005,NaN,3.8,True,False,False,True,False,False,True
2,Lexus,RX 350 RX 350,2022,22372,54598,NaN,NaN,True,False,False,True,False,False,False
3,INFINITI,Q50 Hybrid Sport,2015,88900,15500,354.0,3.5,False,True,False,True,False,False,False
4,Audi,Q3 45 S line Premium Plus,2021,9835,34999,NaN,2.0,True,False,False,True,False,False,False
6,Audi,S3 2.0T Premium Plus,2017,84000,31000,292.0,2.0,True,False,False,True,False,False,False
7,BMW,740 iL,2001,242000,7300,282.0,4.4,True,False,False,True,False,False,False
8,Lexus,RC 350 F Sport,2021,23436,41927,311.0,3.5,True,False,False,True,False,False,False
9,Tesla,Model X Long Range Plus,2020,34000,69950,534.0,NaN,False,False,True,True,False,False,False
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,73897,NaN,NaN,True,False,False,True,False,False,False


 I feel like I need to come up with some better data validation practices. That's for another time though. We'll see if I really messed up when we take a look at the MSE values.

# One-Hot Encode Categorical Data

I'm putting this as it's own section cause I've done one-hot encoding using `pandas`, but not `sklearn`. So let's see if I can get it done (probably gonna be pretty easy). The only categorical column I've kept is `brand`, since the color categories probably don't provide much insight, while there are too many different models in the `model` column to provide useful comparisons (at least I think). However, I think I might have to put this theory to the test.

How about this? We'll add on one more requirement to this project: predict `price` using high and low dimensional data and compare the results. This will be in addition to the requirement of comparing the various missing value imputation methods we are about to explore as well. We'll explore high dimensional data methods in another notebook much later.

In [2449]:
ohe = OneHotEncoder(handle_unknown="ignore", 
                    sparse_output=False).set_output(transform="pandas")
cars = pd.concat([cars,
                  ohe.fit_transform(cars[["brand"]])],
                  axis=1).drop(columns=["brand", "model"])
cars.shape

(3870, 67)

Alright, looks like the dimensionality of the data is still good with just `brand` encoded. Let's move on.

# Missing Value Imputation

I'm in uncharted territory with this kind of stuff, but I have a feeling it might not be too challenging. Just gonna take a lot of background studying. Let's go over some of the options we have.

Here's a [general overview](https://scikit-learn.org/stable/modules/impute.html) for Sklearn native missing value imputation. We'll do a deep dive into each Sklearn option in the `sklearn_missing_val_imputation_methods.ipynb` notebook. After that, we'll do the same with [LightGBM](https://lightgbm.readthedocs.io/en/stable/) in a separate notebook, then I can finally create my own attempt using personally selected features for a custom linear regression model. Each imputation method will be assigned to its own data frame.

Before any of that, let's do a quick double check to see if any additional columns have `NaN` values:

In [2450]:
cols_with_nans = [x for x in cars if cars[x].isnull().sum() > 0]
cars[cols_with_nans].isnull().sum()

horsepower    760
liters        360
dtype: int64

Beyond the methods I'm about to explore, I can also think of a few other ways we could fill in this data even more reliably, but we're here for practice, not the pursuit of the perfect model. That is for another time. Just leaving this note to keep that train of thought in mind. Anyway, let's do the simple stuff now.

Here's a dictionary that will hold the results of each method for the future comparisons:

In [2451]:
imp_datasets = {}

## Scikit: Mean, Median, and Most Frequent Value Imputation

Much to my destain, I think it's important to compare the performance of these more basic missing value imputations to the more advanced methods. This will act as a sort of bonus for familiarizing myself with `scikit`'s syntax and various options.

In [2452]:
# simple_imp_target_cols = ['horsepower', 'liters']

# imp_mean = SimpleImputer(strategy='mean')
# cars_mean = cars.copy()
# cars_mean[simple_imp_target_cols] = imp_mean.fit_transform(cars[simple_imp_target_cols])

# imp_median = SimpleImputer(strategy='median')
# cars_median = cars.copy()
# cars_median[simple_imp_target_cols] = imp_median.fit_transform(cars[simple_imp_target_cols])

# imp_most_freq = SimpleImputer(strategy='most_frequent')
# cars_freq = cars.copy()
# cars_freq[simple_imp_target_cols] = imp_most_freq.fit_transform(cars[simple_imp_target_cols])

# imp_datasets["scikit_mean"] = cars_mean
# imp_datasets["scikit_median"] = cars_median
# imp_datasets["scikit_most_frequent"] = cars_freq


I think that could've been written much more neatly. Let's try again

In [2453]:
def simple_impute(data: pd.DataFrame, strategy: str, target_cols: list[str]):
    data = data.copy()
    imp_simple = SimpleImputer(strategy=strategy)
    data[target_cols] = imp_simple.fit_transform(data[target_cols])
    return data

simple_imp_target_cols = ['horsepower', 'liters']
imp_datasets["scikit_mean"] = simple_impute(cars, "mean", simple_imp_target_cols).round(2) # No need for all those decimals 
imp_datasets["scikit_median"] = simple_impute(cars, "median", simple_imp_target_cols)
imp_datasets["scikit_most_frequent"] = simple_impute(cars, "most_frequent", simple_imp_target_cols)

Beautiful. Makes me so happy.

## Scikit: Iterative Imputer

I'll be referring to [this video](https://www.youtube.com/watch?v=m_qKhnaYZlc) for some help as the Sklearn documentation doesn't give me much confidence.

I'm also noticing I'm using like 3 different names for Sklearn interchangeably. I don't know how that happened.

In [2454]:
imp_it = IterativeImputer()
imp_datasets["scikit_it"] = pd.DataFrame(imp_it.fit_transform(cars), columns=cars.columns, index=cars.index)
imp_datasets["scikit_it"].head(10)

,model_year,milage,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,...,brand_Rolls-Royce,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo
0,2013.0,51000.0,10300.0,300.000000,3.700000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021.0,34742.0,38005.0,341.239373,3.800000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022.0,22372.0,54598.0,343.255047,3.630839,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015.0,88900.0,15500.0,354.000000,3.500000,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021.0,9835.0,34999.0,300.635233,2.000000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017.0,84000.0,31000.0,292.000000,2.000000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2001.0,242000.0,7300.0,282.000000,4.400000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021.0,23436.0,41927.0,311.000000,3.500000,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2020.0,34000.0,69950.0,534.000000,4.066865,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,2021.0,27608.0,73897.0,379.346556,3.089196,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Scikit: KNN Imputer

In [2455]:
imp_knn = KNNImputer(n_neighbors=5) # Could test out different values, but I don't wanna go down that rabbit hole for this application
imp_datasets["scikit_knn"] = pd.DataFrame(imp_knn.fit_transform(cars), columns=cars.columns, index=cars.index)
imp_datasets["scikit_knn"].head(10)

,model_year,milage,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,...,brand_Rolls-Royce,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo
0,2013.0,51000.0,10300.0,300.0,3.70,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021.0,34742.0,38005.0,324.6,3.80,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022.0,22372.0,54598.0,391.2,3.78,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015.0,88900.0,15500.0,354.0,3.50,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021.0,9835.0,34999.0,224.0,2.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017.0,84000.0,31000.0,292.0,2.00,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2001.0,242000.0,7300.0,282.0,4.40,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021.0,23436.0,41927.0,311.0,3.50,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2020.0,34000.0,69950.0,534.0,4.08,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10,2021.0,27608.0,73897.0,458.6,4.68,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


That was suspiciously easy... I hope the above code is correct.

I also wanted to take a look at imputing missing values using decision trees somehow, as discussed in [this paper](https://www.sciencedirect.com/science/article/pii/S2772662223001819), but I think I should look into that another time. Certainly worth keeping note of for now. As I tend to frequently say, let's move on.

**Note from future Christian:** Turns out LightGBM uses a form of decision tree modeling!

# LightGBM

Before you get started, refer to `lightgbm_missing_val_imputation_methods.ipynb` for how to proceed.

Alright. Let's do it. Major credit to [this guy](https://www.youtube.com/watch?v=KWrZ59nLLSg&t=618s).

In [2456]:
# Function could definitely be more secure but we're not working with the categorical data anyway
def lgbm_impute(data: pd.DataFrame):
    data = data.copy()
    cols_with_nans = [x for x in data if data[x].isnull().sum() > 0]
    for col in cols_with_nans:
        data[f"{col}_is_nan"] = data[col].isnull().astype(int)
        train = data[f"{col}_is_nan"]

    return data
    
imp_datasets["lgbm_reg"] = lgbm_impute(cars)

train = cars[~cars.isna()]
train[["horsepower", "liters"]].head(10)

,horsepower,liters
0,300.0,3.7
1,NaN,3.8
2,NaN,NaN
3,354.0,3.5
4,NaN,2.0
6,292.0,2.0
7,282.0,4.4
8,311.0,3.5
9,534.0,NaN
10,NaN,NaN


After some further research, I think this is a topic for another time. Taking too much background research to confidently make it happen.

Let's instead add one more dataset to compare. This one will simply remove all rows with missing values.

In [2457]:
del imp_datasets["lgbm_reg"]
imp_datasets["no_nan"] = cars.dropna()
del cars

Now that we have cleaned the data, encoded the categorical features, and imputed missing values with various methods, it's finally time to actually create a linear regression model.

# Linear Regression

Let's review our options for performing linear regression.
* Ordinary Least Squares
* Ridge Regression
* Lasso

## Split the Data

In [2458]:
# Another function that ideally would be more restrictive for its inputs,
# but we'll worry about that in a more serious project
def imp_data_splits(data: dict):
    split_data = {}
    for imp_type, imp_data in data.items():
        X, y = imp_data.drop(["price"], axis=1), imp_data["price"]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=0, shuffle=True
        )
        split_data[imp_type] = {
            "X_train": X_train, "X_test": X_test,
            "y_train": y_train, "y_test": y_test
        }
    return split_data

split_datasets = imp_data_splits(imp_datasets)

print(split_datasets.keys())
print(split_datasets["scikit_mean"].keys())

dict_keys(['scikit_mean', 'scikit_median', 'scikit_most_frequent', 'scikit_it', 'scikit_knn', 'no_nan'])
dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


## OLS Regression

I was originally attempting to make this function all-encompassing for the different regression methods. Unfortunately, I found out I should practice them individually before attempting that. For the scope of this project it's probably unnecessary anyway.

In [2459]:
def ols_regression(data: dict):
    imp_types = list(data.keys())
    results = {}
    for imp_type in imp_types:
        X_train = data[imp_type]["X_train"]
        X_test = data[imp_type]["X_test"]
        y_train = data[imp_type]["y_train"]
        y_test = data[imp_type]["y_test"]

        ols = linear_model.LinearRegression()
        ols.fit(X_train, y_train)
        y_pred = ols.predict(X_test)

        results[imp_type] = {
            "model": ols,
            "mse": mean_squared_error(y_test, y_pred), 
            "r2_score": r2_score(y_test, y_pred),
            "predictions": y_pred
        }
    return results

ols_results = ols_regression(split_datasets)

## Ridge Regression

In [2460]:
def ridge_regression(data: dict):
    imp_types = list(data.keys())
    results = {}
    for imp_type in imp_types:
        X_train = data[imp_type]["X_train"]
        X_test = data[imp_type]["X_test"]
        y_train = data[imp_type]["y_train"]
        y_test = data[imp_type]["y_test"]

        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        ridge = linear_model.Ridge(alpha=0.1)
        ridge.fit(X_train, y_train)
        y_pred = ridge.predict(X_test)

        results[imp_type] = {
            "model": ridge,
            "mse": mean_squared_error(y_test, y_pred), 
            "r2_score": r2_score(y_test, y_pred),
            "predictions": y_pred
        }

    return results
        
ridge_results = ridge_regression(split_datasets)
ridge_results["scikit_mean"]["predictions"]

array([ 2.45531905e+04,  1.38098960e+05,  4.34457319e+04,  4.08465385e+04,
        5.20228556e+04,  2.20420033e+05,  4.77378026e+04,  5.66294726e+04,
        2.32311943e+04,  5.55650414e+04,  6.56384272e+04,  4.86293394e+04,
        3.90858315e+04,  3.67709037e+04,  5.27915831e+04,  4.66706909e+04,
        3.12111965e+04,  3.59520450e+04,  5.79827628e+04,  6.10033103e+04,
        3.88987836e+04,  7.60910845e+04,  6.58145887e+04,  1.10652178e+04,
       -1.33518978e+04, -1.16575049e+04,  4.47972166e+04,  3.78137950e+04,
        3.60226476e+04,  6.65899676e+04,  2.83239741e+04,  2.03059211e+04,
       -8.03854592e+03,  4.03748411e+05,  6.24296516e+04,  3.99825729e+04,
        2.90624333e+04,  3.54941919e+04,  1.65316217e+04,  7.13986575e+04,
       -1.14265422e+04,  2.21606282e+04,  4.03939153e+04,  6.55138202e+04,
        2.23906684e+04,  1.85457424e+04,  3.40109049e+04,  1.33521135e+04,
        3.82403844e+05,  4.83827887e+04,  4.82336078e+04,  6.50269426e+04,
        1.18169373e+04,  

# Lasso